In [1]:
import nltk                                # Python library for NLP
from nltk.corpus import twitter_samples    # sample Twitter dataset from NLTK
import matplotlib.pyplot as plt
from helper_funtion import clean_text,process_tweet,build_freqs
import numpy as np    

In [2]:
twitter_samples

<TwitterCorpusReader in 'C:\\Users\\8086f\\AppData\\Roaming\\nltk_data\\corpora\\twitter_samples'>

In [3]:
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [4]:
print(len(all_positive_tweets))
print(len(all_negative_tweets))

5000
5000


In [5]:
# split the data into two pieces, one for training and one for testing (validation set)
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

# avoid assumptions about the length of all_positive_tweets
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

In [6]:
train_x[5]

'@BhaktisBanter @PallaviRuhail This one is irresistible :)\n#FlipkartFashionFriday http://t.co/EbZ0L2VENM'

In [7]:
for i in range(len(train_x)):
  train_x[i]=clean_text(train_x[i])
for i in range(len(test_x)):
  test_x[i]=clean_text(test_x[i])

In [12]:
train_x[0]

'followfriday franceint pkuchli milipolpari top engag member commun week'

In [13]:
pro_train_x=process_tweet(train_x)
pro_test_y=process_tweet(test_x)

In [14]:
pro_train_x[0]

['followfriday',
 'franceint',
 'pkuchli',
 'milipolpari',
 'top',
 'engag',
 'member',
 'commun',
 'week']

In [15]:
freqs=build_freqs(pro_train_x, train_y)
first_key = next(iter(freqs))
print(first_key, freqs[first_key])
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

('followfriday', 1.0) 23
type(freqs) = <class 'dict'>
len(freqs) = 16527


In [16]:
freqs

{('followfriday', 1.0): 23,
 ('franceint', 1.0): 1,
 ('pkuchli', 1.0): 1,
 ('milipolpari', 1.0): 1,
 ('top', 1.0): 29,
 ('engag', 1.0): 7,
 ('member', 1.0): 13,
 ('commun', 1.0): 27,
 ('week', 1.0): 72,
 ('lambja', 1.0): 1,
 ('hey', 1.0): 59,
 ('jame', 1.0): 7,
 ('odd', 1.0): 2,
 ('pleas', 1.0): 81,
 ('call', 1.0): 27,
 ('contact', 1.0): 4,
 ('centr', 1.0): 1,
 ('abl', 1.0): 6,
 ('assist', 1.0): 1,
 ('mani', 1.0): 28,
 ('thank', 1.0): 515,
 ('despiteoffici', 1.0): 1,
 ('listen', 1.0): 14,
 ('last', 1.0): 39,
 ('night', 1.0): 56,
 ('bleed', 1.0): 2,
 ('amaz', 1.0): 40,
 ('track', 1.0): 5,
 ('scotland', 1.0): 2,
 ('side', 1.0): 14,
 ('congrat', 1.0): 15,
 ('yeaaaah', 1.0): 1,
 ('yippppi', 1.0): 1,
 ('accnt', 1.0): 2,
 ('verifi', 1.0): 2,
 ('rqst', 1.0): 1,
 ('succeed', 1.0): 1,
 ('got', 1.0): 60,
 ('blue', 1.0): 8,
 ('tick', 1.0): 1,
 ('mark', 1.0): 2,
 ('fb', 1.0): 4,
 ('profil', 1.0): 2,
 ('day', 1.0): 184,
 ('bhaktisbant', 1.0): 16,
 ('pallaviruhail', 1.0): 8,
 ('one', 1.0): 91,
 ('ir

In [17]:
def lookup(freqs, word, label):
    """
    Given a dictionary of frequencies, a word, and a label (1 for positive, 0 for negative),
    return the count of the (word, label) pair.
    """
    return freqs.get((word, label), 0)

In [18]:
def train_naive_bayes(freqs,train_x,train_y):
    '''here first gonna get prob of pos word and neg word'''


    loglikelihood = {}
    logprior = 0
    # calculate V, the number of unique words in the vocabulary
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)

    # calculate N_pos, N_neg, V_pos, V_neg
    N_pos = N_neg = V_pos = V_neg = 0
    for pair in freqs.keys():
        if pair[1]>0:
            # increment the count of unique positive words by 1
            V_pos+=1
            # increment the number of positive words by the count for this (word,label) pair
            N_pos += freqs[pair]
        else:
            # increment the count of unique negative words by 1
            V_neg += 1

            # increment the number of negative words by the count for this (word,label) pair
            N_neg += freqs[pair]

    D = len(train_y)

    D_pos = (len(list(filter(lambda x: x > 0, train_y))))
    D_neg = (len(list(filter(lambda x: x <= 0, train_y))))

     # Calculate logprior
    logprior = np.log(D_pos) - np.log(D_neg)
    for word in vocab:

        #   get the positive and negative frequency of the word
        freq_pos = lookup(freqs,word,1)
        freq_neg = lookup(freqs,word,0)
        # calculate the probability that each word is positive, and negative
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)

        # calculate the log likelihood of the word
        loglikelihood[word] = np.log(p_w_pos/p_w_neg)

    ### END CODE HERE ###

    return logprior, loglikelihood

In [19]:
logprior, loglikelihood = train_naive_bayes(freqs, pro_train_x, train_y)
print(logprior)
print(len(loglikelihood))

0.0
14308


In [20]:

def preprocess_tweet(tweet):
    """
    Preprocess the tweet by tokenizing and normalizing.
    This function can be adjusted based on the preprocessing steps used during training.
    """
    # Tokenize the tweet into words (simple split by spaces, can be enhanced)
    wordss=clean_text(tweet)
    words=wordss.split()
    return words

In [28]:
def predict_by_model(tweet,logprior,loglikelihood):
    words = preprocess_tweet(tweet)
    # initialize probability to zero
    p = 0

    # add the logprior
    p += logprior

    for word in words:

        # check if the word exists in the loglikelihood dictionary
        if word in loglikelihood:
            # add the log likelihood of that word to the probability
            p += loglikelihood[word]


    return p


In [31]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    """
    Input:
        test_x: A list of tweets
        test_y: the corresponding labels for the list of tweets
        logprior: the logprior
        loglikelihood: a dictionary with the loglikelihoods for each word
    Output:
        accuracy: (# of tweets classified correctly)/(total # of tweets)
    """
    accuracy = 0  # return this properly

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    y_hats = []
    for tweet in test_x:
        # if the prediction is > 0
        if predict_by_model(tweet, logprior, loglikelihood) > 0:
            # the predicted class is 1
            y_hat_i = 1
        else:
            # otherwise the predicted class is 0
            y_hat_i = 0

        # append the predicted class to the list y_hats
        y_hats.append(y_hat_i)

    # error is the average of the absolute values of the differences between y_hats and test_y
    error = np.mean(np.absolute(y_hats-test_y))

    # Accuracy is 1 minus the error
    accuracy = 1-error

    ### END CODE HERE ###

    return accuracy


In [32]:

print("Naive Bayes accuracy = %0.4f" %
      (test_naive_bayes(test_x, test_y, logprior, loglikelihood)))

Naive Bayes accuracy = 0.7700


In [34]:
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    # print( '%s -> %f' % (tweet, naive_bayes_predict(tweet, logprior, loglikelihood)))
    p = predict_by_model(tweet, logprior, loglikelihood)
#     print(f'{tweet} -> {p:.2f} ({p_category})')
    print(f'{tweet} -> {p:.2f}')

I am happy -> 2.08
I am bad -> -1.24
this movie should have been great. -> 1.97
great -> 2.08
great great -> 4.16
great great great -> 6.23
great great great great -> 8.31


In [36]:
my_tweet = 'you are bad :('
predict_by_model(my_tweet, logprior, loglikelihood)

-1.2387204058495098

In [27]:
tweet='i deeply sad'
p = test_the_model(tweet, logprior, loglikelihood)
print('The expected output is', p)
# p = test_the_model(tweet, logprior, loglikelihood)
# print('The expected output is', p)

The expected output is -3.0202233749103757


In [37]:
def get_ratio(freqs, word):
    '''
    Input:
        freqs: dictionary containing the words

    Output: a dictionary with keys 'positive', 'negative', and 'ratio'.
        Example: {'positive': 10, 'negative': 20, 'ratio': 0.5}
    '''
    pos_neg_ratio = {'positive': 0, 'negative': 0, 'ratio': 0.0}
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    # use lookup() to find positive counts for the word (denoted by the integer 1)
    pos_neg_ratio['positive'] = lookup(freqs,word,1)

    # use lookup() to find negative counts for the word (denoted by integer 0)
    pos_neg_ratio['negative'] = lookup(freqs,word,0)

    # calculate the ratio of positive to negative counts for the word
    pos_neg_ratio['ratio'] = (pos_neg_ratio['positive'] + 1)/(pos_neg_ratio['negative'] + 1)
    ### END CODE HERE ###
    return pos_neg_ratio

In [43]:

get_ratio(freqs, 'never')

{'positive': 31, 'negative': 42, 'ratio': 0.7441860465116279}

In [45]:
my_tweet = 'I am happy because I am progressing :)'

p = test_the_model(my_tweet, logprior, loglikelihood)
print(p)

1.3276870378252548
